In [2]:
import gurobipy as gp

# Q1

### Simple way 

In [3]:
#gp.setParam("OutputFlag",1)
model=gp.Model()
x1=model.addVar(lb=100,ub=1000)
x2=model.addVar(lb=100,ub=1000)
x3=model.addVar(lb=100,ub=1000)
x4=model.addVar()
x4.lb=100
x4.ub=1000
model.setObjective(16*x1+15*x2+20*x3+22*x4,gp.GRB.MAXIMIZE)
model.addConstr(2*x1+x2+3*x3+4*x4<=3000)
model.addConstr(3*x1+4*x2+5*x3+6*x4<=6000)
model.optimize()
print("problem status is", model.status)
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("x1",x1.X)
print("x1",x2.X)
print("x1",x3.X)
print("x1",x4.X)
print("optimal value",model.objval)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
problem status is 2
x1 860.0
x1 580.0
x1 100.0
x1 100.0
optimal value 26660.0


In [4]:
help(model.addVars)


Help on method addVars in module gurobipy:

addVars(*indexes, **kwargs) method of gurobipy.Model instance
    addVars(*indexes, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS,
               name="")
    
    Add variables in bulk, given one or more lists or integers that serve as
    indexes for the variables.  Returns a dictionary of Var objects, indexed by
    the values (or tuples of values) from the index set.
    
    The optional parameters (lb, ub, obj, vtype, name) work similar
    to the addVar() method, with the following exceptions:
    1. The parameter name is required (ex: vtype=GRB.BINARY)
    2. You can specify the value as a scalar, a list or a dictionary.  For a scalar,
       the value will be used for all variables; for a list, the values must be
       in the same order as the index set; for a dictionary, they must be indexed
       by the variable index.
    3. If you specify a scalar string for name, the variable name will be
       subscripted automatica

### Better way-using dictionaries

In [5]:
#Data
products=["Ray","Zapper","Squire","Soaker"]
resources=["plastic","time"]
profit={"Ray":16
        ,"Zapper":15
        ,"Squire":20,
        "Soaker":22}
resAvail={"plastic":3000,"time":6000}
resUse={("Ray","plastic"):2,("Ray","time"):3,
    ("Zapper","plastic"):1,("Zapper","time"):4,
    ("Squire","plastic"):3,("Squire","time"):5,
    ("Soaker","plastic"):4,("Soaker","time"):6
    }

#Model
model=gp.Model()
pvars=model.addVars(products,lb=100,ub=1000,obj=profit,name="products")
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstrs(gp.quicksum(resUse[p,r]*pvars[p] for p in products)<=resAvail[r] 
                 for r in resources)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.666000000e+04
optimal value 26660.0

    Variable            X 
-------------------------
products[Ray]          860 
products[Zapper]          580 
products[Squire]          100 
products[Soaker]          100 


### Using for loops

In [6]:
#Data
products=["Ray","Zapper","Squire","Soaker"]
resources=["plastic","time"]
profit={"Ray":16
        ,"Zapper":15
        ,"Squire":20,
        "Soaker":22}
resAvail={"plastic":3000,"time":6000}
resUse={("Ray","plastic"):2,("Ray","time"):3,
    ("Zapper","plastic"):1,("Zapper","time"):4,
    ("Squire","plastic"):3,("Squire","time"):5,
    ("Soaker","plastic"):4,("Soaker","time"):6
    }

#Model
model=gp.Model()
pvars={}
for p in products:
    pvars[p]=model.addVar(lb=100,ub=1000,obj=profit[p],name=p)
model.ModelSense=gp.GRB.MAXIMIZE

for r in resources:
    lhs=0
    for p in products:
        lhs+= resUse[p,r]*pvars[p] 
    model.addConstr(lhs<=resAvail[r]) 


model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.666000000e+04
optimal value 26660.0

    Variable            X 
-------------------------
         Ray          860 
      Zapper          580 
      Squire          100 
      Soaker          100 


### Alternative (also good) way-using arrays

A matter of taste which one you prefer. Good to know both as sometimes one may be more convenient than the other. 

In [7]:
import numpy as np
profit = np.array([16,15, 20, 22])
res_use=np.array([[2,1,3,4],[3,4,5,6]])
res_avail = np.array([3000, 6000])   
n = len(profit)                                    # number of variables
m = len(res_avail)                                 # number of constraints
assert res_use.shape == (m, n)
model=gp.Model("Q1")
pvars=model.addVars(n,name="products",lb=100,ub=1000)
model.setObjective(gp.quicksum(profit[i]*pvars[i] for i in range(n)),gp.GRB.MAXIMIZE)
cons=model.addConstrs(gp.quicksum(res_use[i,j]*pvars[j] for j in range(n))<=res_avail[i]
                     for i in range(m))
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

ModuleNotFoundError: No module named 'numpy'

# Q2

In [8]:
makeCosts=[6,13,20]
buyACosts=[12,15,21]
buyBCosts=[11,16,23]
laborUse=[1,2,3]
demands=[100,80,70]
aProdLimit=60
laborAvail=200
n=3 #vars num
model=gp.Model()
M=model.addVars(n,name="make",obj=makeCosts)
A=model.addVars(n,name="BuyA",obj=buyACosts)
B=model.addVars(n,name="BuyB",obj=buyBCosts)
#by default minimization
model.addConstrs(M[i]+A[i]+B[i]==demands[i] for i in range(n)) #sattisfy demands
model.addConstr(gp.quicksum(laborUse[i]*M[i] for i in range(n))<=laborAvail)
model.addConstr(gp.quicksum(A[i] for i in range(n))<=aProdLimit)
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 9 columns and 15 nonzeros
Model fingerprint: 0xba04db3c
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0400000e+03   1.769142e+01   0.000000e+00      0s
       4    3.2200000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.220000000e+03
optimal value 3220.0

    Variable            X 
-------------------------
     make[0]          100 
     make[1]           50 
     BuyA[2]           60 
     BuyB[1]           30 
     BuyB[2]           10 


# Q3

In [ ]:
Bonds=["A","B","C","D","E"]
n=len(Bonds)
returns=[.095,.08,.09,.09,.09]
longMaturity=[True,False,True,True,False]
highRisk=[1,0,0,1,1]
taxFree=[1,1,0,1,0]

totalInvestment=100000
shortTermMinRatio=.5
lowRiskRatio=.55
taxFreeMinRatio=.3
taxFreeRetMinRatio=.4

model=gp.Model()
x=model.addVars(n,name=Bonds,obj=returns)
model.ModelSense=gp.GRB.MAXIMIZE
model.addConstr(gp.quicksum(x[i] for i in range(n))==totalInvestment)
model.addConstr(gp.quicksum(x[i] for i in range(n) if not longMaturity[i])
                >=totalInvestment*shortTermMinRatio)
model.addConstr(gp.quicksum(x[i] for i in range(n) if not highRisk[i])
                >=totalInvestment*lowRiskRatio)
model.addConstr(gp.quicksum(x[i] for i in range(n) if taxFree[i])
                >=totalInvestment*taxFreeMinRatio)
model.addConstr(gp.quicksum(returns[i]*x[i] for i in range(n) if taxFree[i])
                >=taxFreeRetMinRatio*gp.quicksum(returns[i]*x[i] for i in range(n)))

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")
#model.write("Q3.lp")
#with open("Q3.lp",'r') as f:
#    print(f.read())

# Q4

In [ ]:
groves=['A','B','C']
plants=['D','E','F']
distances={('A','D'):21,('A','E'):50,('A','F'):40,
           ('B','D'):35,('B','E'):30,('B','F'):22,
           ('C','D'):55,('C','E'):20,('C','F'):25}
costPerMile=1.5
capacities={'D':2000,'E':6000,'F':2250}
supplies={'A':2750,'B':4000,'C':3000}

model=gp.Model()
x=model.addVars(distances,name='flow')
model.setObjective(costPerMile*gp.quicksum(distances[g,p]*x[g,p] 
                                           for g in groves
                                           for p in plants))
model.addConstrs(gp.quicksum(x[g,p] for p in plants)==supplies[g] 
                 for g in groves)
model.addConstrs(gp.quicksum(x[g,p] for g in groves)<=capacities[p] 
                 for p in plants)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q5

In [ ]:
productionCosts=[250,253,255,253,250,255]
demands=[1000,4500,6000,4500,3500,2000]
maxProduction=[4000,3500,4000,4500,4000,2500]
periods=6
minInventory=1500
maxInventory=6000
startingInventory=1800
inventoryCost=4

model=gp.Model()
P=model.addVars(periods,name="P",obj=productionCosts,
               lb=[0.5*i for i in maxProduction],ub=maxProduction)
I=model.addVars(periods,name="I",obj=inventoryCost,lb=1500,ub=6000)
model.addConstr(I[0]==startingInventory+P[0]-demands[0])
model.addConstrs(I[t]==I[t-1]+P[t]-demands[t] for t in range(1,periods))
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

# Q6

In [ ]:
investments=["A","B","C","D"]
available={"A":[1,2,3,4,5,6],"B":[1,3,5],"C":[1,4],"D":[1]}
maturity={"A":1,"B":2,"C":3,"D":6}
yields={"A":1.018,"B":1.035,"C":1.058,"D":1.11}
yearCommitments=[0,250000,0,250000,0,300000]
model=gp.Model()
myVars={}
for inv in investments:
    myVars[inv]=model.addVars(available[inv],name=inv)
obj=0
for inv in investments:
    obj+=myVars[inv][1]
model.setObjective(obj)

for t in range(1,7):
    lhs=0
    for inv in investments:
        if (t-maturity[inv]+1) in available[inv]: #investments maturing at end of t
            lhs+=yields[inv]*myVars[inv][t-maturity[inv]+1]
    rhs=yearCommitments[t-1] #-1 because counter starts from 0
    for inv in investments:
        if (t+1) in available[inv]:
            rhs+=myVars[inv][t+1]
    model.addConstr(lhs==rhs)

model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

#model.write("Q6.lp")
#with open("Q6.lp",'r') as f:
#       print(f.read())
        